In [4]:
import os,sys
sys.path.append('/home/tanmay/JHU/project/deep-mediation/final_code/src')
import tensorflow as tf
import importlib
import auxiliaryfunctions
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import numpy as np
from scipy.stats import zscore,norm
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import os
import pandas as pd
import random as python_random

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,make_scorer,mean_absolute_error,r2_score

from keras import backend as K

from keras.callbacks import LearningRateScheduler, ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam, SGD

In [11]:
early_stopping = True
epochs = 1000
batch_size = 4
use_dynamic_LR=False
algo = 'svr' 
patience = 50
result_path = '/home/tanmay/JHU/project/deep-mediation/final_code/results/simulation-1'
num_subs = [100,500,1000]
num_iters = 20
num_runs = 100
new_shape = [28,28]

In [12]:
def create_empty_df(num_runs,num_iters):
    # Creates an empty dataFrame
    a = np.empty((num_runs,num_iters))
    a[:] = np.nan
    dataFrame = None
    parameters = ['alpha0', 'beta0','alpha','beta','gamma']
    for params in parameters:
        iter = ['iter_'+str(i) for i in range(num_iters)]
        pdindex = pd.MultiIndex.from_product([[params], iter],
                                             names=['parameters', 'runs']) 
        frame = pd.DataFrame(a, columns = pdindex,index = range(0,num_runs))
        dataFrame = pd.concat([dataFrame,frame],axis=1)
    return dataFrame

def simulate_mediation(df,M,params_df,model,algo,num_subs,batch_size,epochs,tune,n,n_runs,iterations):
    """
    """
    early_stopping=True
    # Initialize z with some phi
    print("Using intial random model parameters")
#     if algo == 'svr':
#         z = svr.predict(M.flatten())
    z = model.predict(M) 
    try:
        z = np.concatenate(z)
    except:
        pass
    # plot_scatter(z,df.m,labels=['z','m'])
    
    for i in range(0,iterations):
        print('Starting iteration... %s'%(i+1))
        #Check for sign change
        if np.corrcoef(z,df.Y)[0,1] < 0 :
            z = z*(-1)
        # Check for scaling
        z= zscore(z)

        lm = smf.ols(formula='z ~ X', data=df).fit()
        alpha0 = lm.params.loc['Intercept']
        alph = lm.params.loc['X'] 

        lm = smf.ols(formula='Y ~ z + X', data=df).fit()
        beta0 = lm.params.Intercept 
        bet = lm.params.z
        gam = lm.params.X 
        resid_std = np.std(lm.resid)
        e = df.Y - beta0 - (df.X*gam)
        h = alpha0 + df.X*alph
        d = (((bet*e)+h)/((bet**2)+1))
        d = zscore(d)
        
        if tune:
            if algo == 'shallow':
                if early_stopping:
                    es = EarlyStopping(monitor='val_loss', mode='min', verbose=0,patience=50)
                    history = model.fit(M,d,batch_size=batch_size,epochs=epochs,verbose=1,callbacks=[es],
                                      shuffle=True,validation_split = 0.3)
                else:
                    history = model.fit(M,d,batch_size=batch_size,epochs=epochs,verbose=1,
                                  shuffle=True,validation_split = 0.3)
                z = model.predict(M)
                
            elif algo == 'svr':
                    model = auxiliaryfunctions.create_svr_model(M,d,tune=False)
                    z = model.predict(M)
            elif algo=='deep':
                print("WIP")

        
        params_df.loc[n_runs]['alpha0','iter_'+str(i)]=alpha0
        params_df.loc[n_runs]['beta0','iter_'+str(i)]=beta0
        params_df.loc[n_runs]['beta','iter_'+str(i)]=bet
        params_df.loc[n_runs]['gamma','iter_'+str(i)]=gam
        params_df.loc[n_runs]['alpha','iter_'+str(i)]=alph

        try:
            z = np.concatenate(z)
        except:
            pass
       
    return params_df,z


In [13]:
# os.environ["CUDA_VISIBLE_DEVICES"]="0"
for n in num_subs:
    start = 0
    output_df = create_empty_df(num_runs,num_iters)
    for runs in range(start,num_runs):
        K.clear_session()
        print("Running simulation for num subs %s starting with %s runs"%(n,runs+1))
    # simulate the dataset
        df,mediator,z = auxiliaryfunctions.simulate_dataset(n,resize= True,
                                          new_shape=new_shape,visualize=False,algo=algo,alpha=0.2)
    # create a model 
        if algo == 'svr':
            model = auxiliaryfunctions.create_svr_model(mediator,df.m)
        elif algo== 'shallow':
            input_shape = (new_shape[0],new_shape[1]*4,1)
            model = auxiliaryfunctions.create_shallow_model2D(input_shape=input_shape)
        else:
            input_shape = (new_shape[0],new_shape[1]*4,1)
            model = auxiliaryfunctions.create_deep_model2D(input_shape=input_shape)

        params_df,z_final = simulate_mediation(df,mediator,output_df,model,algo,n,batch_size,epochs,
                                      True,n,runs,num_iters)
        params_df.to_excel(os.path.join(result_path,'simulation-1-num-subs-'+str(n)+'-'+algo+'.xlsx'))

Running simulation for num subs 1000 starting with 1 runs
Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   59.0s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  3.4min finished


[LibSVM]Using intial random model parameters
Starting iteration... 1
[LibSVM]Starting iteration... 2
[LibSVM]Starting iteration... 3
[LibSVM]Starting iteration... 4
[LibSVM]Starting iteration... 5
[LibSVM]Starting iteration... 6
[LibSVM]Starting iteration... 7
[LibSVM]Starting iteration... 8
[LibSVM]Starting iteration... 9
[LibSVM]Starting iteration... 10
[LibSVM]Starting iteration... 11
[LibSVM]Starting iteration... 12
[LibSVM]Starting iteration... 13
[LibSVM]Starting iteration... 14
[LibSVM]Starting iteration... 15
[LibSVM]Starting iteration... 16
[LibSVM]Starting iteration... 17
[LibSVM]Starting iteration... 18
[LibSVM]Starting iteration... 19
[LibSVM]Starting iteration... 20
[LibSVM]Running simulation for num subs 1000 starting with 2 runs
Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  3.7min finished


[LibSVM]Using intial random model parameters
Starting iteration... 1
[LibSVM]Starting iteration... 2
[LibSVM]Starting iteration... 3
[LibSVM]Starting iteration... 4
[LibSVM]Starting iteration... 5
[LibSVM]Starting iteration... 6
[LibSVM]Starting iteration... 7
[LibSVM]Starting iteration... 8
[LibSVM]Starting iteration... 9
[LibSVM]Starting iteration... 10
[LibSVM]Starting iteration... 11
[LibSVM]Starting iteration... 12
[LibSVM]Starting iteration... 13
[LibSVM]Starting iteration... 14
[LibSVM]Starting iteration... 15
[LibSVM]Starting iteration... 16
[LibSVM]Starting iteration... 17
[LibSVM]Starting iteration... 18
[LibSVM]Starting iteration... 19
[LibSVM]Starting iteration... 20
[LibSVM]Running simulation for num subs 1000 starting with 3 runs
Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  3.6min finished


[LibSVM]Using intial random model parameters
Starting iteration... 1
[LibSVM]Starting iteration... 2
[LibSVM]Starting iteration... 3
[LibSVM]Starting iteration... 4
[LibSVM]Starting iteration... 5
[LibSVM]Starting iteration... 6
[LibSVM]Starting iteration... 7
[LibSVM]Starting iteration... 8
[LibSVM]Starting iteration... 9
[LibSVM]Starting iteration... 10
[LibSVM]Starting iteration... 11
[LibSVM]Starting iteration... 12
[LibSVM]Starting iteration... 13
[LibSVM]Starting iteration... 14
[LibSVM]Starting iteration... 15
[LibSVM]Starting iteration... 16
[LibSVM]Starting iteration... 17
[LibSVM]Starting iteration... 18
[LibSVM]Starting iteration... 19
[LibSVM]Starting iteration... 20
[LibSVM]Running simulation for num subs 1000 starting with 4 runs
Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.1min


KeyboardInterrupt: 